In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(),
                       r'Homework\\homework_04\pandas_challange\Instructions\HeroesOfPymoli'))
	print(os.getcwd())
except:
	pass

In [2]:
from IPython import get_ipython

In [3]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
get_ipython().run_line_magic('matplotlib', 'notebook')
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"
# Read Purchasing File and store into Pandas data frame
md_df = pd.read_csv(file_to_load)
md_df.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
total_players = len(md_df['SN'].unique())
print(f'Total Players: {total_players}')

Total Players: 576


In [5]:
# 
Do Item ID's and Item Names have a 1:1 map?
print(len(md_df['Item ID'].unique()) == len(md_df['Item Name'].unique()))
## We find that some unique items have more than one Item ID
## These are not mere duplicates the prices have changed with the new ID's
# Perhaps we have found evidence of a sale or promotional pricing
md_df['Item ID'] = md_df['Item ID'].apply(str)
# This little bit collects all instances of unique values in each column per Item Name
# It provides a "history" of each item
df = md_df.groupby('Item Name').agg(lambda x: x.unique().tolist()).reset_index()
df[df['Gender'].str.len() > 1]
## This may be a usefull form for analysis later

False


,Item Name,Purchase ID,SN,Age,Gender,Item ID,Price
1,"Aetherius, Boon of the Blessed","[214, 248, 369, 575, 631]","[Iaralsuir44, Isursuir31, Heuli25, Sondadar26,...","[23, 44, 9, 27]","[Male, Female]",[137],[3.39]
3,Alpha,"[673, 699, 765]","[Idacal95, Tyaelly53, Irith83]","[30, 17, 18]","[Male, Female]",[130],[2.07]
4,"Alpha, Oath of Zeal","[352, 496, 754]","[Ilarin91, Tyeuduen32, Pheosurllorin41]","[22, 23]","[Male, Female]",[79],[4.05]
8,Arcane Gem,"[55, 434, 442]","[Phaelap26, Irillo49, Zontibe81]","[25, 20, 21]","[Female, Male]",[84],[3.79]
10,Azurewrath,"[34, 82, 518, 551, 632, 703]","[Sondim73, Haerithp41, Yalostiphos68, Iri67, F...","[19, 16, 29, 20, 17, 21]","[Male, Other / Non-Disclosed]",[160],[4.4]
...,...,...,...,...,...,...,...
171,Winter's Bite,"[42, 229, 259, 400, 615, 663, 725, 759]","[Aidaillodeu39, Undosia27, Haellysu29, Ilarin9...","[18, 17, 20, 22, 25]","[Male, Female]",[72],[3.77]
173,Woeful Adamantite Claymore,"[142, 221, 257]","[Adastirin33, Ealrion88, Yasur35]","[35, 30, 26]","[Female, Male]",[175],[4.48]
174,Wolf,"[52, 90, 141, 394, 479, 550, 661, 775]","[Aelin32, Undosian34, Zhisrisu83, Lassassasda3...","[20, 21, 10, 12]","[Male, Female]",[60],[3.54]
175,"Wolf, Promise of the Moonwalker","[31, 137, 305, 429, 481, 565]","[Farusrian86, Iri67, Siallylis44, Aiduesu86, S...","[37, 20, 22, 7]","[Male, Female]",[179],[4.48]


# Purchase summary

In [6]:
purch_cols = ['Item Name', 'Price']
def purchase_summary(dataframe, cols):
	ui_count = len(dataframe[cols[0]].unique())
	avg_price = dataframe[cols[1]].mean()
	sales_count = len(dataframe[cols[1]])
	total_sales = dataframe[cols[1]].sum()
	summary = {'Unique Items': ui_count,
			   'Average Price': avg_price,
			   'Total Purchases': sales_count,
			   'Total Sales': total_sales}
	summary_df = pd.DataFrame(summary, index=[0])
	summary_df['Average Price'] =  summary_df['Average Price'].map('${:,.2f}'.format)
	summary_df['Total Sales'] =  summary_df['Total Sales'].map('${:,.2f}'.format)
	return summary_df
purchase_summary(md_df, purch_cols)


,Unique Items,Average Price,Total Purchases,Total Sales
0,179,$3.05,780,"$2,379.77"


# Gender Demographics
 This method is agnostic with regards to number unique genders reported

In [7]:
demographics = md_df.groupby(['SN', 'Gender']).agg(lambda x: x.unique().tolist()).reset_index()
gen_cols = 'Gender'
def demo_summary(dataframe, cols):
	
	demo_df = dataframe[cols]
	gen_tally = Counter(demo_df)
	demo_summary = pd.DataFrame(gen_tally, index=[0])
	demo_summary = demo_summary
	demo_percentage = demo_summary.div(demo_summary.sum(axis=1), axis=0).applymap('{:,.2%}'.format)
	demo_summary = pd.concat([demo_summary, demo_percentage], ignore_index=True)
	return demo_summary

summary_df = demo_summary(demographics, gen_cols)
summary_df


,Male,Female,Other / Non-Disclosed
0,484,81,11
1,84.03%,14.06%,1.91%


 ## Purchasing Analysis (Gender)

In [8]:
pag_cols = ['Gender', 'Price']
calc_cols = ['Avg Price', 'Purchase Count', 'Total Sales']
# Here's where i started having a difficult time trying to implement this the way i wanted.
# I had a realization that at least for right now jupyter notebooks are for rapid prototyping 
# and swift data exploration. IE check out the seaborn stuff I read up on in a book later on in the notebook.
def gp_summary(dataframe, sort_cols, stat_cols):
	#summary_df = demo_summary(dataframe, 'Gender')
	gp_df = dataframe[sort_cols].groupby(sort_cols[0]).mean().applymap('${:,.2f}'.format)
	gp_df.rename(columns = {'Price':'Avg Price'}, inplace = True)
	gp_df['Purchase Count'] = dataframe[sort_cols].groupby('Gender').count()
	gp_df['Total Sales'] = dataframe[sort_cols].groupby('Gender').sum()
	#For some reason the line below doesn't calculate what it's supposed too or doesn't store what it's supposed to. I'm not sure which.
	#gp_df['Per Person Per Gender'] = (gp_df['Total Sales']/summary_df.iloc[0])
	return gp_df
# It turns out i was feeding it the wrong value. Give it Unique Names and it behaves correctly
gap_summary = gp_summary(md_df, pag_cols, calc_cols)
gap_summary['Per Person Per Gender'] = gap_summary['Total Sales'].astype('float')/summary_df.iloc[0].astype('float')
gap_summary['Per Person Per Gender'] = gap_summary['Per Person Per Gender'].map('${:,.2f}'.format)
gap_summary
# After spending alot of time modularizing this I realized that at this level
# Notebooks are purpose built for prototyping and not meant to be generic.
#From here on out I went direct.

,Avg Price,Purchase Count,Total Sales,Per Person Per Gender
Gender,,,,
Female,$3.20,113,361.94,$4.47
Male,$3.02,652,1967.64,$4.07
Other / Non-Disclosed,$3.35,15,50.19,$4.56


 ## Purchasing Analysis (Age)

In [9]:
# These bins took some fiddling
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
#If you don't want to type out intervals
labels = [f'{i+1} - {i+5}' for i in bins] 
labels[0] = '<10'
labels[-1] = '40+'
del labels[-2]
#print(len(bins))
#print(len(labels))
#print(labels)


In [10]:
# * Categorize the existing players using the age bins. Hint: use pd.cut()
md_df['Age Group'] = pd.cut(md_df['Age'], bins=bins, labels=labels,include_lowest=True)
md_df.tail()
age_df = md_df[['SN', 'Age Group']].groupby('SN').first()
#age_sum = pd.DataFrame(age_df, index=[1])
age_tally = age_df['Age Group'].value_counts()
age_per = age_tally.div(age_tally.sum(), axis=0).map('{:,.2%}'.format)

age_summary = pd.DataFrame({'Total Count': age_tally,
							'Percentage': age_per})
age_summary 


,Total Count,Percentage
20 - 24,258,44.79%
15 - 19,107,18.58%
25 - 29,77,13.37%
30 - 34,52,9.03%
35 - 39,31,5.38%
10 - 14,22,3.82%
<10,17,2.95%
40+,12,2.08%


 ## Purchasing Analysis (Age)

 * Bin the purchase_data data frame by age

In [11]:
pba_df = md_df[['Age Group', 'Price']].groupby('Age Group').count()
pba_df['Avg Purchase'] = md_df[['Age Group', 'Price']].groupby('Age Group').mean()
pba_df['Total Sales'] = md_df[['Age Group', 'Price']].groupby('Age Group').sum()
pba_df['Sales by Group Per Person'] = (pba_df['Total Sales'].astype('float')/age_summary['Total Count'].astype('float'))
pba_df[['Avg Purchase', 'Total Sales', 'Sales by Group Per Person']]= pba_df[['Avg Purchase', 'Total Sales', 'Sales by Group Per Person']].applymap('${:,.2f}'.format)
pba_df.rename(columns={'Price': 'Total Purchases'}, inplace=True)
pba_df


,Total Purchases,Avg Purchase,Total Sales,Sales by Group Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10 - 14,28,$2.96,$82.78,$3.76
15 - 19,136,$3.04,$412.89,$3.86
20 - 24,365,$3.05,"$1,114.06",$4.32
25 - 29,101,$2.90,$293.00,$3.81
30 - 34,73,$2.93,$214.00,$4.12
35 - 39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


# Top Spenders

In [12]:
top_shop_df = md_df.groupby(['SN', 'Gender', 'Age']).agg(lambda x: x.unique().tolist()).reset_index()
top_shop_df = top_shop_df[['SN', 'Price']]
top_shop_df.set_index('SN')
top_shop_df['Purchases'] = top_shop_df['Price'].apply(len)
top_shop_df['Price'] = top_shop_df['Price'].apply(np.sum)
top_shop_df['Avg Purchase Price'] = top_shop_df['Price']/top_shop_df['Purchases']
top_shop_df.nlargest(columns='Avg Purchase Price', n=10)
top_shop_df.rename(columns={'Price': 'Total Purchases'}, inplace=True)
top_ten_play = top_shop_df.nlargest(columns='Total Purchases', n = 10)
top_ten_play[['Total Purchases', 'Avg Purchase Price']] = top_shop_df[['Total Purchases', 'Avg Purchase Price']].applymap('${:,.2f}'.format)
top_ten_play


,SN,Total Purchases,Purchases,Avg Purchase Price
360,Lisosia93,$18.96,5,$3.79
246,Idastidru52,$15.45,4,$3.86
106,Chamjask73,$13.83,3,$4.61
275,Iral74,$13.62,4,$3.40
281,Iskadarya95,$13.10,3,$4.37
250,Ilarin91,$12.70,3,$4.23
228,Ialallo29,$11.84,3,$3.95
531,Tyidaim51,$11.83,3,$3.94
313,Lassilsala30,$11.51,3,$3.84
88,Chadolyla44,$11.46,3,$3.82


 ## Most Popular Items

In [13]:
top_item_df = md_df[['Item ID', 'Item Name', 'Price', 'Purchase ID']].groupby(['Item ID', 'Item Name']).agg(lambda x: x.unique().tolist()).reset_index().set_index('Item ID')
top_item_df['Purchase Count'] = top_item_df['Purchase ID'].apply(len)
top_item_df['Total Item Sales'] = top_item_df['Purchase Count']*(top_item_df['Price'].apply(np.sum))
top_item_df.nlargest(columns='Purchase Count', n=10)

# * Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


,Item Name,Price,Purchase ID,Purchase Count,Total Item Sales
Item ID,,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",[4.23],"[25, 61, 62, 72, 222, 464, 580, 627, 658, 706,...",12,50.76
108,"Extraction, Quickblade Of Trembling Hands",[3.53],"[0, 56, 157, 414, 483, 498, 558, 640, 678]",9,31.77
145,Fiery Glass Crusader,[4.58],"[132, 207, 329, 438, 474, 572, 683, 741, 769]",9,41.22
82,Nirvana,[4.9],"[18, 88, 202, 240, 247, 274, 504, 522, 666]",9,44.10
103,Singed Scalpel,[4.35],"[175, 373, 455, 460, 476, 634, 660, 714]",8,34.80
19,"Pursuit, Cudgel of Necromancy",[1.02],"[239, 344, 361, 528, 582, 654, 657, 773]",8,8.16
34,Retribution Axe,[2.22],"[65, 84, 350, 449, 559, 576, 604, 685]",8,17.76
37,"Shadow Strike, Glory of Ending Hope",[3.16],"[29, 69, 197, 244, 314, 322, 357, 745]",8,25.28
59,"Lightning, Etcher of the King",[4.23],"[91, 154, 179, 349, 376, 439, 480, 503]",8,33.84


# Most Profitable Item

In [14]:
top_item_df.nlargest(columns='Total Item Sales', n=10)

,Item Name,Price,Purchase ID,Purchase Count,Total Item Sales
Item ID,,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",[4.23],"[25, 61, 62, 72, 222, 464, 580, 627, 658, 706,...",12,50.76
82,Nirvana,[4.9],"[18, 88, 202, 240, 247, 274, 504, 522, 666]",9,44.10
145,Fiery Glass Crusader,[4.58],"[132, 207, 329, 438, 474, 572, 683, 741, 769]",9,41.22
92,Final Critic,[4.88],"[2, 252, 273, 277, 712, 722, 767, 768]",8,39.04
103,Singed Scalpel,[4.35],"[175, 373, 455, 460, 476, 634, 660, 714]",8,34.80
59,"Lightning, Etcher of the King",[4.23],"[91, 154, 179, 349, 376, 439, 480, 503]",8,33.84
108,"Extraction, Quickblade Of Trembling Hands",[3.53],"[0, 56, 157, 414, 483, 498, 558, 640, 678]",9,31.77
78,"Glimmer, Ender of the Moon",[4.4],"[118, 206, 263, 616, 636, 687, 692]",7,30.80
72,Winter's Bite,[3.77],"[42, 229, 259, 400, 615, 663, 725, 759]",8,30.16


# Trends and Facts